# Export station coords

Export station coordinates to a file to that it is readily available when we perform interpolation.
This is better than rerunning the query to the mongodb every time, because it's kind of expensive to 
look at 2 years of data to find all the possible stations.

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import csv
import datetime
import os
import pathlib
import pandas as pd
import pymongo

In [ ]:
DATA_DIR = pathlib.Path(os.getenv('DATA_DIR'))

MONGO_URL = 'localhost'
MONGO_PORT = 27017
USERNAME = None
PASSWORD = None
ADMIN_DB = 'admin'
COLLECTION = 'iem'
DB = 'smc01_raw_obs'

In [ ]:
begin_date = datetime.datetime(2019, 1, 1, tzinfo=datetime.timezone.utc)
end_date = datetime.datetime(2021, 1, 1, tzinfo=datetime.timezone.utc)

In [ ]:
end_date - begin_date

In [ ]:
mongo_client = pymongo.MongoClient(host=MONGO_URL, port=MONGO_PORT, username=USERNAME, password=PASSWORD, authSource=ADMIN_DB)
db = mongo_client[DB]
collection = db[COLLECTION]

In [ ]:
query_filter = {
    'valid': {
        '$gte': begin_date,
        '$lt': end_date
}}

In [ ]:
stations = set(collection.distinct('station'))

In [ ]:
station_info = []
for station in stations:
    one_obs = collection.find_one({'station': station})
    station_info.append({
        'station': station,
        'lat': one_obs['lat'],
        'lon': one_obs['lon'],
    })

In [ ]:
with open(DATA_DIR / 'metar_stations.csv', 'w') as csv_file:
    writer = csv.DictWriter(csv_file, ['station', 'lat', 'lon'], )
    writer.writeheader()
    
    for station in sorted(station_info, key=lambda x: x['station']):
        writer.writerow(station)

In [ ]:
!head ${DATA_DIR}/metar_stations.csv